In [ ]:
#save table in a sql server
#change window size to get 20 listings
#keep clicking previous till not possible
import time
import pandas as pd
import re
import seaborn as sb
import datetime as dt

: 

In [16]:
monthword = ['January','February','March','April','May','June','July','August','September','October','November','December']
weekdayword = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
(dt.datetime.now() + dt.timedelta(days=2)).year
now = dt.datetime.now()
daypoint = now + dt.timedelta(days=2)
print(f'{str(daypoint.day)}, {weekdayword[daypoint.weekday()]}, {monthword[daypoint.month]} {str(daypoint.year)}')

28, Saturday, February 2023


In [2]:
df = pd.read_csv('airbnb.csv',index_col=[0])

In [3]:
len(df)

2158

In [ ]:

df['text'][0]

In [4]:
len(df[(df['days booked'] == 0)&(df['city'] == 'Chicago')])

59

In [5]:
df.tail()

,room,text,days booked,date,search position,city,state,reviews,adults,children,infants,pets
2153,49458886,Skip to content Start your searchStaysExpe...,36,2023-01-24 19:23:17.589219,266,Chicago,Illinois,96,2,0,0,0
2154,plus/26251178,Skip to content Start your searchStaysExpe...,7,2023-01-24 19:23:20.455801,267,Chicago,Illinois,390,2,0,0,0
2155,25376091,Skip to content Start your searchStaysExpe...,5,2023-01-24 19:23:22.760447,268,Chicago,Illinois,135,2,0,0,0
2156,770523073506526538,Skip to content Start your searchStaysExpe...,29,2023-01-24 19:23:24.914181,269,Chicago,Illinois,3,2,0,0,0
2157,52918623,Skip to content Start your searchStaysExpe...,14,2023-01-24 19:23:27.055321,270,Chicago,Illinois,50,2,0,0,0


In [6]:
#Cleanliness5.0Accuracy5.0Communication5.0Location5.0Check-in5.0Value4.9

processed = []
errd = []

def try_patterns(text:str,patterns:list[str],num:bool = False):
    for p in patterns:
        try:
            regstring = re.search(p,text).group()
            if num:
                return float(regstring)
            return regstring
        except AttributeError:
            pass
    if num:
        return 0
    return ''

for i in range(len(df)):
    text = df['text'][i]
    State = df['state'][i]
    try:
        a = {
        'guests':try_patterns(text,['\d+(?= guest)'],num=True),#

        'bedrooms':try_patterns(text,['\d+(?= bedroom)'],num=True),#

        'beds':try_patterns(text,['\d+(?= beds)','\d+(?= bed )'],num=True),#

        'bathrooms':try_patterns(text,['[\d.]+(?= bath)','\d+(?= shared bath)','\d+(?= private bath)'],num=True),#

        'price': try_patterns(text,['(?<=\$)\d+(?= per night)'],num=True),#

        'Superhost':text.find('Superhost') > -1,#

        'location':try_patterns(text,[f'[\w ]+(?=, {State})','[\w ]+(?=, United States)']),#

        'rating':try_patterns(text,['\d\.\d+'],num=True), #

        'place type': try_patterns(text,['(?<=Reserve)[\w ]+(?= hosted)','(?<=availability)[\w ]+(?= hosted)','[\w ]+(?= hosted)']),#

        'State':State, #

        'Cleanliness rating':try_patterns(text,['(?<=Cleanliness)\d\.\d+'],num=True),

        'Accuracy rating':try_patterns(text,['(?<=Accuracy)\d\.\d+'],num=True),

        'Communication rating':try_patterns(text,['(?<=Communication)\d\.\d+'],num=True),

        'Location rating':try_patterns(text,['(?<=Location)\d\.\d+'],num=True),

        'Check-in rating':try_patterns(text,['(?<=Check-in)\d\.\d+'],num=True),

        'Value rating':try_patterns(text,['(?<=Value)\d\.\d+'],num=True),
        
        'plus':df['room'][i][0:4] == 'plus'
        }

        if df['reviews'][i] < 3 and a['rating'] != 0:
            print('see')
            a['rating'] = 0
        

        if a['bedrooms'] == 0 & text.find('Studio') > -1:
            a['bedrooms'] = 0.5

        

        processed.append(a)

    except Exception as e:
        errd.append(e)

    
tempdf = pd.DataFrame(processed)

see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see
see


In [7]:
len(errd)

0

In [8]:
len(tempdf)

2158

In [13]:
len(df[df['text'].str.find('Studio') > -1])

153

In [15]:
df.tail()

,room,text,days booked,date,search position,city,state,reviews,adults,children,infants,pets
1614,29877084,Skip to content Start your searchStaysExpe...,24,2023-01-17 10:40:58.387337,266,Chicago,Illinois,197,2,0,0,0
1615,48215227,Skip to content Start your searchStaysExpe...,35,2023-01-17 10:41:00.866310,267,Chicago,Illinois,56,2,0,0,0
1616,690637882075022232,Skip to content Start your searchStaysExpe...,2,2023-01-17 10:41:02.807303,268,Chicago,Illinois,17,2,0,0,0
1617,734871188071815020,Skip to content Start your searchStaysExpe...,73,2023-01-17 10:41:04.910763,269,Chicago,Illinois,5,2,0,0,0
1618,750207467391040511,Skip to content Start your searchStaysExpe...,12,2023-01-17 10:41:06.934881,270,Chicago,Illinois,6,2,0,0,0


In [ ]:
tempdf.head()

In [ ]:
tempdf['plus'].describe()

In [ ]:

tempdf[ (tempdf['reviews'] == 0)]

In [ ]:
len(df[df['room'].str[0:4] == 'plus'])

In [ ]:
df['room'][565]

In [ ]:
re.search('(?<=$)\d+(?= reviews)',df['text'][63]) == None

In [ ]:
df['text'][63]